### Install and import packages

In [1]:
# Import packages
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()
sns.set_palette("tab10")

import random as rnd
import numpy as np
rnd.seed(0)
np.random.seed(0)

import scipy.stats as stats
import bayes_logistic

from utils import *
from cores import generate_prior_and_posterior_samples, sample_laplace_approx

### Set parameters

In [2]:
# Set parameters
# data_x parameters
args = {}
args["seed"] = 1
args["num_feats"] = 2
args["num_data"] = 2
args["num_data_half"] = args['num_data'] // 2

# num_samples
args["num_samples"] = 1000

# weights prior distribution parameters
args["weights_prior_params"] = [[0.0, 0.0], [[1.0, 0.0], [0.0, 1.0]]]

# init sigma used in numerical optimization for laplace approximation
args["laplace_num_iters"] = 1000

### Generate data_x

In [3]:
# Generate data_x
# data_x = stats.uniform.rvs(0, 1, size=(num_data, num_feats), random_state=12345)
data_x = np.array([[0, 1], [1, 0]])
print(data_x)
print(data_x.shape)

[[0 1]
 [1 0]]
(2, 2)


### Sample weights, generate sample y from sample weights and visualize data

In [16]:
### Generate prior and posterior samples

weights_prior_params_list = [
    [[0.0, 0.0], [[1.0, 0.0], [0.0, 1.0]]],
    [[0.0, 0.0], [[4.0, -1.0], [-1.0, 4.0]]],
    [[0.0, 0.0], [[25.0, 0.0], [0.0, 25.0]]],
    [[0.0, 0.0], [[49.0, 0.0], [0.0, 49.0]]],
]

for weights_prior_params in weights_prior_params_list:
    print("\n-> weights_prior_params")
    print(weights_prior_params)
    args["weights_prior_params"] = weights_prior_params
    samples_a_weights_prior, samples_b_weights_prior, samples_a_weights_posterior = \
            generate_prior_and_posterior_samples(data_x, sample_laplace_approx, args)

    samples_a_weights_prior = np.vstack(samples_a_weights_prior)
    samples_b_weights_prior = np.vstack(samples_b_weights_prior)
    samples_a_weights_posterior = np.vstack(samples_a_weights_posterior)
    
    # Maximum mean distance with RBF kernel
    mmd_rbf_prior_a_prior_b = compute_mmd_rbf(samples_a_weights_prior, samples_b_weights_prior)
    mmd_rbf_posterior_a_prior_b = compute_mmd_rbf(samples_a_weights_posterior, samples_b_weights_prior)
    print(f"MMD between prior a and prior b: {mmd_rbf_prior_a_prior_b:0.5f}")
    print(f"MMD between posterior a and prior b: {mmd_rbf_posterior_a_prior_b:0.5f}")
    
    # Wasserstein distance with RBF kernel
    wd_prior_a_prior_b = compute_wasserstein_distance(samples_a_weights_prior, samples_b_weights_prior)
    wd_posterior_a_prior_b = compute_wasserstein_distance(samples_a_weights_posterior, samples_b_weights_prior)
    print(f"Wasserstein distance between prior a and prior b: {wd_prior_a_prior_b:0.5f}")
    print(f"Wasserstein distance between posterior a and prior b: {wd_posterior_a_prior_b:0.5f}")
    
    # Difference between the standard deviations (from true mean) of two samples
    weights_prior_params = args["weights_prior_params"]
    diff_std_prior_a_prior_b = compute_diff_std(samples_a_weights_prior, samples_b_weights_prior, weights_prior_params[0])
    diff_std_posterior_a_prior_b = compute_diff_std(samples_a_weights_posterior, samples_b_weights_prior, weights_prior_params[0])
    print(f"Difference standard deviations between between prior a and prior b: {diff_std_prior_a_prior_b:0.5f}")
    print(f"Difference standard deviations between posterior a and prior b: {diff_std_posterior_a_prior_b:0.5f}")


-> weights_prior_params
[[0.0, 0.0], [[1.0, 0.0], [0.0, 1.0]]]
MMD between prior a and prior b: 0.00128
MMD between posterior a and prior b: 0.00265
Wasserstein distance between prior a and prior b: 0.13764
Wasserstein distance between posterior a and prior b: 0.15529
Difference standard deviations between between prior a and prior b: 0.00717
Difference standard deviations between posterior a and prior b: 0.03827

-> weights_prior_params
[[0.0, 0.0], [[4.0, -1.0], [-1.0, 4.0]]]
MMD between prior a and prior b: 0.00161
MMD between posterior a and prior b: 0.00322
Wasserstein distance between prior a and prior b: 0.27549
Wasserstein distance between posterior a and prior b: 0.35157
Difference standard deviations between between prior a and prior b: 0.02339
Difference standard deviations between posterior a and prior b: 0.28503

-> weights_prior_params
[[0.0, 0.0], [[25.0, 0.0], [0.0, 25.0]]]
MMD between prior a and prior b: 0.00212
MMD between posterior a and prior b: 0.00657
Wasserstei